In [161]:
import mysql.connector as msc
from mysql.connector import Error
import pandas as pd
import tkinter as tk
from tkinter import ttk, Canvas
from datetime import date

In [162]:
### SERVER CONNECTION FUNCTION ###

def server_connection(hostName, userName, userPassword, database):
    connection = None
    try:
        connection = msc.connect(
            host = hostName,
            user = userName,
            password = userPassword,
            db = database,
            autocommit = True
        )
        print("connection successful")
    except Error as error:
        print(f"Error: '{error}'")
    return connection  

In [ ]:
### MAIN METHOD ###

frameList = []
accountEmail = []
currentDate = [date.today().strftime("%Y-%m-%d")]
flight_airline_dropdown = ["Delta Airlines", "Southwest Airlines", "American Airlines", "United Airlines", 
                                    "JetBlue Airways", "Spirit Airlines", "WestJet", "Interjet"]
registerCondition = 0

# need to start sql first and enter your sql password into the third parameter
connection = server_connection("localhost", "root", "Jfhacks123!", "travel_reservation_service")
def main():
    root = tk.Tk()
    root.geometry("500x500")
    root.title("Login")
    root.resizable(height = False, width = False)

    loginSetUp(root)

    root.mainloop()
    
main()

connection successful
mmoss1@travelagency.com


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Serena\anaconda3\envs\CS4400\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\Serena\AppData\Local\Temp/ipykernel_16636/1032079473.py", line 65, in <lambda>
    command = lambda : viewCustomers(root, None))
  File "C:\Users\Serena\AppData\Local\Temp/ipykernel_16636/1555412351.py", line 118, in viewCustomers
    cursor.execute(statement)
UnboundLocalError: local variable 'statement' referenced before assignment


In [163]:
### LOGIN ###

def login(root, email, password):
    print(email)

    cursor = connection.cursor()
    cursor.execute("select * from Accounts natural join Admins")
    for row in cursor.fetchall():
        if (email == row[0] and password == row[3]):
            accountEmail.append(email)
            destroyPage()
            adminHomeSetUp(root)
            break
    cursor.close()
    
    cursor = connection.cursor()
    cursor.execute("select * from Accounts natural join Clients")
    for row in cursor.fetchall():
        if (email == row[0] and password == row[3]):
            accountEmail.append(email)
            destroyPage()
            customerHomeSetUp(root)
            break
    cursor.close()
    
def logout(root):
    del accountEmail[0]
    destroyPage()
    loginSetUp(root)

def backAdmin(root):
    destroyPage()
    adminHomeSetUp(root)

def backCustomer(root):
    destroyPage()
    customerHomeSetUp(root)

In [199]:
### ADMIN VIEWS ###

def viewAirlines(root, execution_string):
    root.title("View Airlines")
    destroyPage()
    
    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)
    
    label = tk.Label(frame, text = "View Airlines", font = ("Comic Sans MS", 20))
    label.place(x = 250, y = 60, anchor = tk.CENTER)
    
    backButton = tk.Button(frame, text = "Back", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : backAdmin(root))
    backButton.place(x = 465, y = 20, anchor = tk.CENTER)
    
    fnameLabel = tk.Label(frame, text = "Name:", font = ("Comic Sans MS", 12))
    fnameLabel.place(x = 170, y = 100, anchor = tk.W)
    
    fnameEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    fnameEntry.place(x = 220, y = 90)
    
    statement = "select * from view_airlines"
    if (execution_string != None):
        statement = execution_string
    cursor = connection.cursor()
    cursor.execute(statement)
    
    data = []
    for row in cursor.fetchall():
        data.append(row)
    cursor.close()
    
    columns = ["Name", "Rating", "Total Flights", "Minimum Flight Cost"]
    df = pd.DataFrame(data, columns = columns)
    
    tree = ttk.Treeview(frame, height = 400)
    tree.place(y = 150)
    
    displayTable(frame, tree, df, columns, 120)
    
    filterButton = tk.Button(frame, text = "Filter", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : filterAirlineViewButton(root, fnameEntry.get()))
    filterButton.place(x = 400, y = 100, anchor = tk.CENTER)
    
    resetButton = tk.Button(frame, text = "Reset", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : resetAirlineViewButton(root))
    resetButton.place(x = 100, y = 100, anchor = tk.CENTER)
    
    frame.pack()
    
def filterAirlineViewButton(root, param):
    statement = "select * from view_airlines"
    if (len(param) > 0):
        statement += " where airline_name like '%" + param + "%'"
    viewAirlines(root, statement)
    
def resetAirlineViewButton(root):
    statement = "select * from view_airlines"
    viewAirlines(root, statement)

def viewAirports(root):
    root.title("View Airports")
    destroyPage()
    
    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)
    
    label = tk.Label(frame, text = "View Airports", font = ("Comic Sans MS", 20))
    label.place(x = 250, y = 60, anchor = tk.CENTER)
    
    backButton = tk.Button(frame, text = "Back", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : backAdmin(root))
    backButton.place(x = 465, y = 20, anchor = tk.CENTER)
    
    cursor = connection.cursor()
    cursor.execute("select Airport_Id, Airport_Name, Time_Zone, concat(Street, ', ', City, ', ', State, ' ', Zip) as 'Address' from Airport")
    
    data = []
    for row in cursor.fetchall():
        data.append(row)
    cursor.close()
    
    columns = ["ID", "Name", "Time Zone", "Address"]
    df = pd.DataFrame(data, columns = columns)
    
    tree = ttk.Treeview(frame, height = 400)
    tree.place(y = 100)
    
    displayTable(frame, tree, df, columns, 120)
    
    frame.pack()
    
def viewCustomers(root, execution_string):
    root.title("View Customers")
    destroyPage()
    
    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)
    
    label = tk.Label(frame, text = "View Customers", font = ("Comic Sans MS", 20))
    label.place(x = 250, y = 60, anchor = tk.CENTER)
    
    backButton = tk.Button(frame, text = "Back", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : backAdmin(root))
    backButton.place(x = 465, y = 20, anchor = tk.CENTER)
    
    fnameLabel = tk.Label(frame, text = "Name:", font = ("Comic Sans MS", 12))
    fnameLabel.place(x = 170, y = 100, anchor = tk.W)
    
    fnameEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    fnameEntry.place(x = 220, y = 90)
    
    statement = "select * from view_customers"
    if (execution_string != None):
        statement = execution_string
    cursor = connection.cursor()
    cursor.execute(statement)
    
    data = []
    for row in cursor.fetchall():
        data.append(row)
    cursor.close()
    
    columns = ["Name", "Average Rating", "Location", "is Owner", "Total Seats Purchased"]
    df = pd.DataFrame(data, columns = columns)
    
    tree = ttk.Treeview(frame, height = 400)
    tree.place(y = 150)
    
    displayTable(frame, tree, df, columns, 96)
    
    filterButton = tk.Button(frame, text = "Filter", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : filterCustViewButton(root, fnameEntry.get()))
    filterButton.place(x = 400, y = 100, anchor = tk.CENTER)
    
    resetButton = tk.Button(frame, text = "Reset", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : resetCustViewButton(root))
    resetButton.place(x = 100, y = 100, anchor = tk.CENTER)
    
    frame.pack()

def filterCustViewButton(root, param):
    statement = "select * from view_customers"
    if (len(param) > 0):
        statement += " where customer_name like '%" + param + "%'"
    print(statement)
    viewCustomers(root, statement)
    
def resetCustViewButton(root):
    statement = "select * from view_customers"
    viewCustomers(root, statement)
    
def viewOwners(root):
    root.title("View Owners")
    destroyPage()
    
    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)
    
    label = tk.Label(frame, text = "View Owners", font = ("Comic Sans MS", 20))
    label.place(x = 250, y = 60, anchor = tk.CENTER)
    
    backButton = tk.Button(frame, text = "Back", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : backAdmin(root))
    backButton.place(x = 465, y = 20, anchor = tk.CENTER)
    
    cursor = connection.cursor()
    cursor.execute("select * from view_owners")
    
    data = []
    for row in cursor.fetchall():
        data.append(row)
    cursor.close()
    
    columns = ["Name", "Average Rating", "Number of Properties Ownered", "Average Property Rating"]
    df = pd.DataFrame(data, columns = columns)
    
    tree = ttk.Treeview(frame, height = 400)
    tree.place(y = 100)
    
    displayTable(frame, tree, df, columns, 120)
    
    frame.pack()

In [165]:
### ADMIN PROCEDURES ###

def scheduleFlight(root):
    root.title("Schedule Flight")
    destroyPage()
    
    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)
    
    titleLabel = tk.Label(frame, text = "Schedule Flight", font = ("Comic Sans MS", 20))
    titleLabel.place(x = 250, y = 60, anchor = tk.CENTER)
    
    #left side entries
    
    fnumLabel = tk.Label(frame, text = "Flight Number:", font = ("Comic Sans MS", 12))
    fnumLabel.place(x = 0, y = 100, anchor = tk.W)
    
    fnumEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    fnumEntry.place(x = 130, y = 90)
    
    airlineLabel = tk.Label(frame, text = "Airline:", font = ("Comic Sans MS", 12))
    airlineLabel.place(x = 0, y = 135, anchor = tk.W)
    
    menuTextType = tk.StringVar()
    menuTextType.set(flight_airline_dropdown[0])
    airlineDrop = tk.OptionMenu(root, menuTextType, *flight_airline_dropdown)
    airlineDrop.place(x = 125, y = 125)
    
    fromLabel = tk.Label(frame, text = "From Airport:", font = ("Comic Sans MS", 12))
    fromLabel.place(x = 0, y = 170, anchor = tk.W)
    
    fromEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    fromEntry.place(x = 130, y = 160)
    
    toLabel = tk.Label(frame, text = "To Airport:", font = ("Comic Sans MS", 12))
    toLabel.place(x = 0, y = 205, anchor = tk.W)
    
    toEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    toEntry.place(x = 130, y = 195)
    
    depTimeLabel = tk.Label(frame, text = "Departure Time:", font = ("Comic Sans MS", 12))
    depTimeLabel.place(x = 0, y = 240, anchor = tk.W)
    
    depTimeEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    depTimeEntry.place(x = 130, y = 230)
    
    arrTimeLabel = tk.Label(frame, text = "Arrival Time:", font = ("Comic Sans MS", 12))
    arrTimeLabel.place(x = 0, y = 275, anchor = tk.W)
    
    arrTimeEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    arrTimeEntry.place(x = 130, y = 265)
    
    #right side entries
    
    fDateLabel = tk.Label(frame, text = "Flight Date:", font = ("Comic Sans MS", 12))
    fDateLabel.place(x = 250, y = 100, anchor = tk.W)
    
    fDateEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    fDateEntry.place(x = 380, y = 90)
    
    costLabel = tk.Label(frame, text = "$ Per Person:", font = ("Comic Sans MS", 12))
    costLabel.place(x = 250, y = 135, anchor = tk.W)
    
    costEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    costEntry.place(x = 380, y = 125)
    
    capLabel = tk.Label(frame, text = "Capacity:", font = ("Comic Sans MS", 12))
    capLabel.place(x = 250, y = 170, anchor = tk.W)
    
    capEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    capEntry.place(x = 380, y = 160)
    
    d1 = currentDate[0]
    dateLabel = tk.Label(frame, text = "Current Date: " + d1, font = ("Comic Sans MS", 12))
    dateLabel.place(x = 250, y = 205, anchor = tk.W)
    
    backButton = tk.Button(frame, text = "Back", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : backAdmin(root))
    backButton.place(x = 400, y = 400, anchor = tk.CENTER)
    
    scheduleButton = tk.Button(frame, text = "Schedule", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : scheduleFlightButton(fnumEntry.get(), menuTextType.get(), fromEntry.get(), 
                                                                  toEntry.get(), depTimeEntry.get(), arrTimeEntry.get(), 
                                                                  fDateEntry.get(), costEntry.get(), capEntry.get(), d1))
    scheduleButton.place(x = 100, y = 400, anchor = tk.CENTER)
    
    frame.pack()

def scheduleFlightButton(Flight_Num, Airline_Name, From_Airport, To_Airport, Departure_Time, 
                         Arrival_Time, Flight_Date, Cost, Capacity, Current_Date):
    cursor = connection.cursor()
    cursor.callproc('schedule_flight', [Flight_Num, Airline_Name, From_Airport, To_Airport, Departure_Time, 
                                        Arrival_Time, Flight_Date, Cost, Capacity, Current_Date])
    cursor.close()
    
def removeFlight(root, importedFlightData):
    root.title("Remove Flight")
    destroyPage()
    
    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)
    
    titleLabel = tk.Label(frame, text = "Remove Flight", font = ("Comic Sans MS", 20))
    titleLabel.place(x = 250, y = 60, anchor = tk.CENTER)
    
    datesLabel = tk.Label(frame, text = "Dates:", font = ("Comic Sans MS", 12))
    datesLabel.place(x = 0, y = 100, anchor = tk.W)
    
    datesLeftEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    datesLeftEntry.place(x = 60, y = 90)
    
    dashLabel = tk.Label(frame, text = "-", font = ("Comic Sans MS", 12))
    dashLabel.place(x = 165, y = 100, anchor = tk.W)
    
    datesRightEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    datesRightEntry.place(x = 180, y = 90)
    
    airlineLabel = tk.Label(frame, text = "Airline:", font = ("Comic Sans MS", 12))
    airlineLabel.place(x = 290, y = 100, anchor = tk.W)
    
    menuTextType = tk.StringVar()
    menuTextType.set("Null")
    airlineDrop = tk.OptionMenu(root, menuTextType, *flight_airline_dropdown)
    airlineDrop.place(x = 360, y = 85)
    
    d1 = currentDate[0]
    dateLabel = tk.Label(frame, text = "Current Date: " + d1, font = ("Comic Sans MS", 12))
    dateLabel.place(x = 0, y = 135, anchor = tk.W)
    
    fnumLabel = tk.Label(frame, text = "Flight Number:", font = ("Comic Sans MS", 12))
    fnumLabel.place(x = 230, y = 135, anchor = tk.W)
    
    fnumEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    fnumEntry.place(x = 350, y = 125)
    
    cursor = connection.cursor()
    cursor.execute("select airline_name, flight_num, flight_date from Flight")
    
    flightData = []
    if (importedFlightData != None):
        flightData = importedFlightData
    
    for row in cursor.fetchall():
        if (importedFlightData == None):
            flightData.append(row)
    cursor.close()
    
    columns = ["Airline", "Number", "Date"]
    df = pd.DataFrame(flightData, columns = columns)
    
    tree = ttk.Treeview(frame, height = 10)
    tree.place(y = 165)

    displayTable(frame, tree, df, columns, 96)
    
    backButton = tk.Button(frame, text = "Back", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : backAdmin(root))
    backButton.place(x = 50, y = 450, anchor = tk.CENTER)
    
    resetButton = tk.Button(frame, text = "Reset", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : resetFlightsButton(flightData, root))
    resetButton.place(x = 200, y = 450, anchor = tk.CENTER)
    
    filterButton = tk.Button(frame, text = "Filter", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : filterFlightsButton(flightData, datesLeftEntry.get(), datesRightEntry.get(), 
                                                                 menuTextType.get(), fnumEntry.get(), tree, frame, root))
    filterButton.place(x = 300, y = 450, anchor = tk.CENTER)
    
    removeButton = tk.Button(frame, text = "Remove", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : removeFlightButton(flightData, tree, tree.selection(), d1, root))
    removeButton.place(x = 450, y = 450, anchor = tk.CENTER)
    
    frame.pack()
    
def resetFlightsButton(flightData, root):
    cursor = connection.cursor()
    cursor.execute("select airline_name, flight_num, flight_date from Flight")
    
    flightData.clear()
    for row in cursor.fetchall():
        flightData.append(row)
    cursor.close()
    removeFlight(root, None)

def filterFlightsButton(flightData, startDate, endDate, airline, num, tree, frame, root):
    flightData.clear()
    cursor = connection.cursor()
    execution_string = "select airline_name, flight_num, flight_date from Flight"
    filterExists = 0;
    if (len(startDate) == 10 or len(endDate) == 10 or airline != "Null" or len(num) > 0):
        filterExists = 1;
        execution_string += " where "
    if (filterExists == 1):
        if (len(startDate) == 10):
            if (len(endDate) == 10):
                execution_string = execution_string + "flight_date between '" + startDate + "'' and ''" + endDate + "'"
                filterExists = 2
            else:
                execution_string = execution_string + "flight_date >= '" + startDate + "'"
                filterExists = 2
        if (len(endDate) == 10 and filterExists == 1):
            execution_string = execution_string + "flight_date <= '" + endDate + "'"
            filterExists = 2
        if (airline != "Null"):
            if (filterExists == 2):
                execution_string += " and "
            execution_string = execution_string + "airline_name = " + '"' + airline + '"'
            filterExists = 3
        if (len(num) > 0):
            if (filterExists == 2 or filterExists == 3):
                execution_string += " and "
            execution_string = execution_string + "flight_num = " + num

    cursor.execute(execution_string)
    for row in cursor.fetchall():
        flightData.append(row)
    cursor.close()
    removeFlight(root, flightData)

def removeFlightButton(flightData, tree, treeSelection, currDate, root):
    num = tree.item(treeSelection)["values"][1]
    name = tree.item(treeSelection)["values"][0]
    cursor = connection.cursor()
    cursor.callproc('remove_flight', [num, name, currDate])
    cursor.close()
    flightData.clear()
    cursor = connection.cursor()
    cursor.execute("select airline_name, flight_num, flight_date from Flight")
    
    for row in cursor.fetchall():
        flightData.append(row)
    cursor.close()
    removeFlight(root, None)

def processDate(root):
    root.title("Process Date")
    destroyPage()
    
    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)
    
    titleLabel = tk.Label(frame, text = "Process Date", font = ("Comic Sans MS", 20))
    titleLabel.place(x = 250, y = 60, anchor = tk.CENTER)
    
    backButton = tk.Button(frame, text = "Back", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : backAdmin(root))
    backButton.place(x = 465, y = 20, anchor = tk.CENTER)
    
    setLabel = tk.Label(frame, text = "Set Current\nSystem Date:", font = ("Comic Sans MS", 14))
    setLabel.place(x = 170, y = 170, anchor = tk.E)
    
    entry = tk.Entry(frame, font = ("Comic Sans MS", 14), width = 20)
    entry.place(x = 180, y = 155)
    
    setButton = tk.Button(frame, text = "Set Date", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : setDateButton(entry.get()))
    setButton.place(x = 390, y = 210, anchor = tk.CENTER)
    
    frame.pack()

def setDateButton(date):
    cursor = connection.cursor()
    cursor.callproc('process_date', [date])
    del currentDate[0]
    currentDate.append(date)
    cursor.close()

In [166]:
### CUSTOMER VIEWS ###
def viewFlights(root, inputData):
    root.title("View Flights")
    destroyPage()
    
    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)
    
    label = tk.Label(frame, text = "View Flights", font = ("Comic Sans MS", 20))
    label.place(x = 250, y = 60, anchor = tk.CENTER)
    
    backButton = tk.Button(frame, text = "Back", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : backCustomer(root))
    backButton.place(x = 465, y = 20, anchor = tk.CENTER)

    seatLabel = tk.Label(frame, text = "Available Seats:", font = ("Comic Sans MS", 12))
    seatLabel.place(x = 135, y = 345, anchor = tk.CENTER)

    seatEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    seatEntry.place(x = 180, y = 330)
    
    data = []
    if inputData == None:
        cursor = connection.cursor()
        selectStatement = "select f.flight_num, f.airline_name, f.from_airport, f.to_airport, f.departure_time, f.arrival_time, f.flight_date, calc_seats_remaining(f.flight_num, f.airline_name), f.cost, f.cost * coalesce(b.num_seats, 0) from flight f, book b where f.airline_name = b.airline_name and f.flight_num = b.flight_num"
        cursor.execute(selectStatement)
    
        for row in cursor.fetchall():
            data.append(row)
        cursor.close()
    else:
        data = inputData
    
    columns = ["ID", "Airline", "From", "To", "Dept. Time", "Arr. Time", "Date", "Avail. Seats", "Cost per Seat", "Total Spent"]
    df = pd.DataFrame(data, columns = columns)
    
    tree = ttk.Treeview(frame, height = 400)
    tree.place(y = 100)
    
    displayTable(frame, tree, df, columns, 120)

    filterButton = tk.Button(frame, text = "Filter", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : filterBySeatsButton(root, seatEntry.get()))
    filterButton.place(x = 450, y = 450, anchor = tk.CENTER)
    
    frame.pack()

def filterBySeatsButton(root, seats):
    if (seats != None and int(seats) > 0):
        cursor = connection.cursor()
        selectStatement = "select f.flight_num, f.airline_name, f.from_airport, f.to_airport, f.departure_time, f.arrival_time, f.flight_date, calc_seats_remaining(f.flight_num, f.airline_name) as seats, f.cost, f.cost * coalesce(b.num_seats, 0) from flight f, book b where f.airline_name = b.airline_name and f.flight_num = b.flight_num and seats >= %(numSeats)s"
        cursor.execute(selectStatement, { numSeats: int(seats) })
        
        data = []
        for row in cursor.fetchall():
            data.append(row)
        cursor.close()
        viewFlights(root, data)
    else:
        viewFlights(root, None)

In [ ]:
### CUSTOMER VIEWS ###

def viewProperties(root):
    root.title("View Properties")
    destroyPage()

    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)
    
    label = tk.Label(frame, text = "View Properties", font = ("Comic Sans MS", 20))
    label.place(x = 250, y = 60, anchor = tk.CENTER)
    
    backButton = tk.Button(frame, text = "Back", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : backCustomer(root))
    backButton.place(x = 465, y = 20, anchor = tk.CENTER)

    minCapLabel = tk.Label(frame, text = "Min: ", font = ("Comic Sans MS", 12))
    minCapLabel.place(x = 150, y = 100, anchor = tk.W)

    minCapEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 5)
    minCapEntry.place(x = 190, y = 90)

    maxCapLabel = tk.Label(frame, text = "Max: ", font = ("Comic Sans MS", 12))
    maxCapLabel.place(x = 250, y = 100, anchor = tk.W)

    maxCapEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 5)
    maxCapEntry.place(x = 290, y = 90)

    statement = "select * from view_properties"
    cursor = connection.cursor()
    cursor.execute(statement)

    data = []
    for row in cursor.fetchall():
        data.append(row)
    cursor.close()

    columns = ["Name", "Average Rating", "Description", "Address", "Capacity", "Cost"]
    df = pd.DataFrame(data, columns = columns)

    tree = ttk.Treeview(frame, height = 400)
    tree.place(y = 150)
    
    displayTable(frame, tree, df, columns, 96)

    frame.pack()

In [167]:
### CUSTOMER PROCEDURES ###
def bookFlight(root):
    root.title("Book Flight")
    frameList[0].pack_forget()
    del frameList[0]
    
    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)
    
    label = tk.Label(frame, text = "Book Flight", font = ("Comic Sans MS", 20))
    label.place(x = 250, y = 60, anchor = tk.CENTER)
    
    backButton = tk.Button(frame, text = "Back", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : backCustomer(root))
    backButton.place(x = 465, y = 20, anchor = tk.CENTER)
    
    cursor = connection.cursor()
    cursor.execute("select airline_name, flight_num, from_airport, to_airport, calc_seats_remaining(flight_num, airline_name), flight_date from flight")
    
    flightData = []
    for row in cursor.fetchall():
        flightData.append(row)
    selectStatement = "select book.airline_name, book.flight_num, num_seats * cost from book, flight where book.flight_num = flight.flight_num and book.airline_name = flight.airline_name and book.customer = %(email)s and book.was_cancelled = 0"
    cursor.execute(selectStatement, { 'email': accountEmail[0] })
    bookingData = []
    for row in cursor.fetchall():
            bookingData.append(row)
    cursor.close()
    
    columns = ["Airline", "Flight Num", "From", "To", "Avail. Seats", "Date"]
    df = pd.DataFrame(flightData, columns = columns)
    
    tree = ttk.Treeview(frame, height = 220)
    tree.place(y = 100)

    displayTable(frame, tree, df, columns, [120, 66, 66, 66, 66, 96])

    seatLabel = tk.Label(frame, text = "Seats to Book:", font = ("Comic Sans MS", 12))
    seatLabel.place(x = 135, y = 345, anchor = tk.CENTER)

    seatEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    seatEntry.place(x = 180, y = 330)

    bookButton = tk.Button(frame, text = "Book Flight", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : bookFlightButton(root, tree, tree.selection(), seatEntry.get()))
    bookButton.place(x = 340, y = 345, anchor = tk.CENTER)

    bookedLabel = tk.Label(frame, text = "Booked Flights:", font = ("Comic Sans MS", 16))
    bookedLabel.place(x = 250, y = 380, anchor = tk.CENTER)

    columnsTwo = ["Airline", "Flight Number", "Amount Spent"]
    dfTwo = pd.DataFrame(bookingData, columns = columnsTwo)
    
    treeTwo = ttk.Treeview(frame, height = 75)
    treeTwo.place(y = 400)

    displayTable(frame, treeTwo, dfTwo, columnsTwo, 160)
    
    frame.pack()

def bookFlightButton(root, tree, treeSelection, seats):
    d1 = currentDate[0]
    if seats != "" and int(seats) > 0:
        cursor = connection.cursor()
        for ind in treeSelection:
            row = tree.item(tree.get_children()[int(ind)])['values']
            cursor.callproc('book_flight', [accountEmail[0], row[1], row[0], seats, d1])
        cursor.close()
        bookFlight(root)

def cancelBookedFlight(root, importedFlightData):
    root.title("Cancel Flight")
    destroyPage()
    
    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)
    
    titleLabel = tk.Label(frame, text = "Cancel Flight", font = ("Comic Sans MS", 20))
    titleLabel.place(x = 250, y = 60, anchor = tk.CENTER)
    
    airlineLabel = tk.Label(frame, text = "Airline:", font = ("Comic Sans MS", 12))
    airlineLabel.place(x = 290, y = 100, anchor = tk.W)
    
    menuTextType = tk.StringVar()
    menuTextType.set("Null")
    airlineDrop = tk.OptionMenu(root, menuTextType, *flight_airline_dropdown)
    airlineDrop.place(x = 360, y = 85)
    
    d1 = currentDate[0]
    dateLabel = tk.Label(frame, text = "Current Date: " + d1, font = ("Comic Sans MS", 12))
    dateLabel.place(x = 0, y = 135, anchor = tk.W)
    
    fnumLabel = tk.Label(frame, text = "Flight Number:", font = ("Comic Sans MS", 12))
    fnumLabel.place(x = 230, y = 135, anchor = tk.W)
    
    fnumEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    fnumEntry.place(x = 350, y = 125)
    
    flightData = []
    if (importedFlightData != None):
        flightData = importedFlightData
    else: 
        cursor = connection.cursor()
        selectStatement = "select book.airline_name, book.flight_num, flight.flight_date from book, flight where book.airline_name = flight.airline_name and book.flight_num = flight.flight_num and customer = %(email)s and was_cancelled = 0"
        cursor.execute(selectStatement, { 'email': accountEmail[0] })
        for row in cursor.fetchall():
            flightData.append(row)
        cursor.close()
    
    columns = ["Airline", "Number", "Date"]
    df = pd.DataFrame(flightData, columns = columns)
    
    tree = ttk.Treeview(frame, height = 10)
    tree.place(y = 165)

    displayTable(frame, tree, df, columns, 96)
    
    backButton = tk.Button(frame, text = "Back", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : backCustomer(root))
    backButton.place(x = 50, y = 450, anchor = tk.CENTER)
    
    resetButton = tk.Button(frame, text = "Reset", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : cancelBookedFlight(root, None))
    resetButton.place(x = 200, y = 450, anchor = tk.CENTER)
    
    filterButton = tk.Button(frame, text = "Filter", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : filterBookingsButton(flightData, menuTextType.get(), fnumEntry.get(), tree, frame, root))
    filterButton.place(x = 300, y = 450, anchor = tk.CENTER)
    
    removeButton = tk.Button(frame, text = "Remove", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : removeBookingButton(flightData, tree, tree.selection(), d1, root))
    removeButton.place(x = 450, y = 450, anchor = tk.CENTER)
    
    frame.pack()

def filterBookingsButton(flightData, airline, num, tree, frame, root):
    flightData.clear()
    cursor = connection.cursor()
    execution_string = "select book.airline_name, book.flight_num, flight.flight_date from book, flight where book.airline_name = flight.airline_name and book.flight_num = flight.flight_num and customer = %(email)s and was_cancelled = 0"
    if (airline != "Null"):
        execution_string += " and "
        execution_string = execution_string + "book.airline_name = " + '"' + airline + '"'
    if (len(num) > 0):
        execution_string += " and "
        execution_string = execution_string + "book.flight_num = " + num

    cursor.execute(execution_string, { 'email': accountEmail[0] })
    for row in cursor.fetchall():
        flightData.append(row)
    cursor.close()
    cancelBookedFlight(root, flightData)

def removeBookingButton(flightData, tree, treeSelection, currDate, root):
    num = tree.item(treeSelection)["values"][1]
    name = tree.item(treeSelection)["values"][0]
    cursor = connection.cursor()
    cursor.callproc('cancel_flight_booking', [accountEmail[0], num, name, currDate])
    cursor.close()
    cancelBookedFlight(root, None)

def reserveProperty(root):
    root.title("Reserve Property")
    destroyPage()
    
    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)
    
    titleLabel = tk.Label(frame, text = "Reserve Property", font = ("Comic Sans MS", 20))
    titleLabel.place(x = 250, y = 60, anchor = tk.CENTER)
    
    datesLabel = tk.Label(frame, text = "Dates:", font = ("Comic Sans MS", 12))
    datesLabel.place(x = 10, y = 85, anchor = tk.W)
    
    startDateEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    startDateEntry.place(x = 60, y = 70)
    
    hyphenLabel = tk.Label(frame, text = "-", font = ("Comic Sans MS", 12))
    hyphenLabel.place(x = 160, y = 85, anchor = tk.W)
    
    endDateEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    endDateEntry.place(x = 180, y = 70)
    
    d1 = currentDate[0]
    dateLabel = tk.Label(frame, text = "Current Date: " + d1, font = ("Comic Sans MS", 12))
    dateLabel.place(x = 300, y = 85, anchor = tk.W)
    
    backButton = tk.Button(frame, text = "Back", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : backCustomer(root))
    backButton.place(x = 465, y = 20, anchor = tk.CENTER)
    
    cursor = connection.cursor()
    cursor.execute("select property_name, owner_email, capacity from property")
    
    propertyData = []
    for row in cursor.fetchall():
        propertyData.append(row)
    selectStatement = "select r.property_name, r.start_date, r.end_date, p.cost * (r.end_date - r.start_date) from reserve r, property p where r.property_name = p.property_name and r.owner_email = p.owner_email and customer = %(email)s"
    cursor.execute(selectStatement, { 'email': accountEmail[0] })
    reservationData = []
    for row in cursor.fetchall():
            reservationData.append(row)
    cursor.close()
    
    columns = ["Property Name", "Owner Email", "Capacity"]
    df = pd.DataFrame(propertyData, columns = columns)
    
    tree = ttk.Treeview(frame, height = 220)
    tree.place(y = 100)

    displayTable(frame, tree, df, columns, 160)
    
    guestsLabel = tk.Label(frame, text = "Number of Guests:", font = ("Comic Sans MS", 12))
    guestsLabel.place(x = 135, y = 345, anchor = tk.CENTER)

    guestsEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 10)
    guestsEntry.place(x = 200, y = 330)

    reserveButton = tk.Button(frame, text = "Reserve", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                          command = lambda : reservePropertyButton(root, tree, tree.selection(), startDateEntry.get(), endDateEntry.get(), guestsEntry.get(), d1))
    reserveButton.place(x = 340, y = 345, anchor = tk.CENTER)

    bookedLabel = tk.Label(frame, text = "Reserved Properties:", font = ("Comic Sans MS", 16))
    bookedLabel.place(x = 250, y = 380, anchor = tk.CENTER)

    columnsTwo = ["Property Name", "Start Date", "End Date", "Amount Spent"]
    dfTwo = pd.DataFrame(reservationData, columns = columnsTwo)
    
    treeTwo = ttk.Treeview(frame, height = 75)
    treeTwo.place(y = 400)

    displayTable(frame, treeTwo, dfTwo, columnsTwo, 120)
    
    frame.pack()

def reservePropertyButton(root, tree, treeSelection, startDate, endDate, numGuests, currDate):
    owner = tree.item(treeSelection)["values"][1]
    name = tree.item(treeSelection)["values"][0]
    cursor = connection.cursor()
    cursor.callproc('reserve_property', [name, owner, accountEmail[0], startDate, endDate, numGuests, currDate])
    cursor.close()
    reserveProperty(root)

In [168]:
### REGISTER ###
def register(root):
    root.title("Register")
    destroyPage()

    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)
    
    # basic info
    
    titleLabel = tk.Label(frame, text = "Register", font = ("Comic Sans MS", 20))
    titleLabel.place(x = 250, y = 60, anchor = tk.CENTER)
    
    fnameLabel = tk.Label(frame, text = "First Name:", font = ("Comic Sans MS", 12))
    fnameLabel.place(x = 0, y = 100, anchor = tk.W)
    
    fnameEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 15)
    fnameEntry.place(x = 130, y = 90)
    
    lnameLabel = tk.Label(frame, text = "Last Name:", font = ("Comic Sans MS", 12))
    lnameLabel.place(x = 0, y = 135, anchor = tk.W)
    
    lnameEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 15)
    lnameEntry.place(x = 130, y = 125)
    
    emailLabel = tk.Label(frame, text = "Email:", font = ("Comic Sans MS", 12))
    emailLabel.place(x = 0, y = 170, anchor = tk.W)
    
    emailEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 15)
    emailEntry.place(x = 130, y = 160)
    
    passwordLabel = tk.Label(frame, text = "Password:", font = ("Comic Sans MS", 12))
    passwordLabel.place(x = 0, y = 205, anchor = tk.W)
    
    passwordEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 15)
    passwordEntry.place(x = 130, y = 195)
    
    confirmLabel = tk.Label(frame, text = "Confirm:", font = ("Comic Sans MS", 12))
    confirmLabel.place(x = 0, y = 240, anchor = tk.W)
    
    confirmEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 15)
    confirmEntry.place(x = 130, y = 230)
    
    phoneLabel = tk.Label(frame, text = "Phone number:", font = ("Comic Sans MS", 12))
    phoneLabel.place(x = 0, y = 275, anchor = tk.W)
    
    phoneEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 15)
    phoneEntry.place(x = 130, y = 265)
    
    register = tk.Button(frame, text = "Register", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7)
    register.place(x = 375, y = 400, anchor = tk.CENTER)
    
    back = tk.Button(frame, text = "Back", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                                  command = lambda : logout(root))
    back.place(x = 375, y = 435, anchor = tk.CENTER)

    # cutomer info
    cardNumLabel = tk.Label(frame, text = "Card Number:", font = ("Comic Sans MS", 12))
    cardNumEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 15)
    cvvLabel = tk.Label(frame, text = "CVV:", font = ("Comic Sans MS", 12))
    cvvEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 15)
    expLabel = tk.Label(frame, text = "Exp:", font = ("Comic Sans MS", 12))
    expEntry = tk.Entry(frame, font = ("Comic Sans MS", 12), width = 15)
    
    # button
    
    customerButton = tk.Button(frame, text = "Customer", font = ("Comic Sans MS", 10), fg = "white", bg = "grey", width = 30,
                                  command = lambda : registerCustomer(customerButton, ownerButton, register,
                                                                      fnameEntry, lnameEntry, emailEntry,
                                                                      passwordEntry, confirmEntry, phoneEntry,
                                                                     cardNumLabel, cardNumEntry, cvvLabel,
                                                                      cvvEntry, expLabel, expEntry))
    customerButton.place(x = 125, y = 330, anchor = tk.CENTER)
    
    ownerButton = tk.Button(frame, text = "Owner", font = ("Comic Sans MS", 10), fg = "white", bg = "grey", width = 30,
                                  command = lambda : registerOwner(customerButton, ownerButton, register,
                                                                      fnameEntry, lnameEntry, emailEntry,
                                                                      passwordEntry, confirmEntry, phoneEntry,
                                                                     cardNumLabel, cardNumEntry, cvvLabel,
                                                                      cvvEntry, expLabel, expEntry))
    ownerButton.place(x = 375, y = 330, anchor = tk.CENTER)
    
    frame.pack()

def registerCustomer(customerButton, ownerButton, register,
                     fnameEntry, lnameEntry, emailEntry,
                     passwordEntry, confirmEntry, phoneEntry,
                     cardNumLabel, cardNumEntry, cvvLabel,
                     cvvEntry, expLabel, expEntry):
    customerButton.configure(bg = "black")
    ownerButton.configure(bg = "grey")
    register.configure(command = lambda : registerCustomerButton(emailEntry.get(), fnameEntry.get(), lnameEntry.get(),
                                                                passwordEntry.get(), confirmEntry.get(), phoneEntry.get(),
                                                                cardNumEntry.get(), cvvEntry.get(), expEntry.get()))
    
    global registerCondition
    
    if (registerCondition == 0):
        cardNumLabel.place(x = 0, y = 375, anchor = tk.W)
        cardNumEntry.place(x = 130, y = 365)
        cvvLabel.place(x = 0, y = 410, anchor = tk.W)
        cvvEntry.place(x = 130, y = 400)
        expLabel.place(x = 0, y = 445, anchor = tk.W)
        expEntry.place(x = 130, y = 435)
    registerCondition = 1
    
def registerOwner(customerButton, ownerButton, register,
                     fnameEntry, lnameEntry, emailEntry,
                     passwordEntry, confirmEntry, phoneEntry,
                     cardNumLabel, cardNumEntry, cvvLabel,
                     cvvEntry, expLabel, expEntry):
    customerButton.configure(bg = "grey")
    ownerButton.configure(bg = "black")
    register.configure(command = lambda : registerOwnerButton(emailEntry.get(), fnameEntry.get(), lnameEntry.get(),
                                                                passwordEntry.get(), confirmEntry.get(), phoneEntry.get()))
    
    global registerCondition
    
    if (registerCondition == 1):
        cardNumLabel.place(x = -100, y = -100, anchor = tk.W)
        cardNumEntry.place(x = -100, y = -100)
        cvvLabel.place(x = -100, y = -100, anchor = tk.W)
        cvvEntry.place(x = -100, y = -100)
        expLabel.place(x = -100, y = -100, anchor = tk.W)
        expEntry.place(x = -100, y = -100)
        
    registerCondition = 0

    
def registerCustomerButton(email, fname, lname, password, confirm, phone, card, cvv, exp):
    if (password == confirm and password != "" and fname != "" and lname != "" and email != "" and
       phone != "" and card != "" and cvv != "" and exp != ""):
        try:
            cursor = connection.cursor()
            cursor.callproc('register_customer', [email, fname, lname, password, phone, card, cvv, exp, ""])
            cursor.close()
            print("register complete")
        except:
            print("invalid")
    else:
        print("invalid parameters")
    
def registerOwnerButton(email, fname, lname, password, confirm, phone):
    if (password == confirm and password != "" and fname != "" and lname != "" and email != "" and phone != ""):
        try:
            cursor = connection.cursor()
            cursor.callproc('register_owner', [email, fname, lname, password, phone])
            cursor.close()
            print("register complete")
        except:
            print("invalid")
    else:
        print("invalid parameters")

In [193]:
### FRAME SETUP ###

def loginSetUp(root):
        root.title("Login")
        frame = tk.Frame(root, height = 500, width = 500)
        frameList.append(frame)

        logLabel = tk.Label(frame, text = "Login", font = ("Comic Sans MS", 20))
        logLabel.place(x = 250, y = 100, anchor = tk.CENTER)

        emailLabel = tk.Label(frame, text = "Email:", font = ("Comic Sans MS", 14))
        emailLabel.place(x = 170, y = 170, anchor = tk.E)

        passLabel = tk.Label(frame, text = "Password:", font = ("Comic Sans MS", 14))
        passLabel.place(x = 170, y = 220, anchor = tk.E)

        emailEntry = tk.Entry(frame, font = ("Comic Sans MS", 14), width = 20)
        emailEntry.place(x = 180, y = 155)

        passEntry = tk.Entry(frame, font = ("Comic Sans MS", 14), width = 20)
        passEntry.place(x = 180, y = 205)

        loginButton = tk.Button(frame, text = "Login", font = ("Comic Sans MS", 12), fg = "white", bg = "black", width = 8,
                               command = lambda : login(root, emailEntry.get(), passEntry.get()))
        loginButton.place(x = 250, y = 280, anchor = tk.CENTER)

        registerLabel = tk.Label(frame, text = "Not signed up?", font = ("Comic Sans MS", 10))
        registerLabel.place(x = 250, y = 320, anchor = tk.CENTER)

        registerButton = tk.Button(frame, text = "Register", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 7,
                                  command = lambda : register(root))
        registerButton.place(x = 250, y = 350, anchor = tk.CENTER)

        frame.pack()

def adminHomeSetUp(root):
    root.title("Admin Home")
    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)

    homeLabel = tk.Label(frame, text = "Admin Home", font = ("Comic Sans MS", 20))
    homeLabel.place(x = 250, y = 100, anchor = tk.CENTER)

    scheduleFlightButton = tk.Button(frame, text = "Schedule\nFlight", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8, 
                                command = lambda : scheduleFlight(root))
    scheduleFlightButton.place(x = 125, y = 180, anchor = tk.CENTER)

    removeFlightButton = tk.Button(frame, text = "Remove\nFlight", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8, 
                                   command = lambda : removeFlight(root, None))
    removeFlightButton.place(x = 250, y = 180, anchor = tk.CENTER)

    processDateButton = tk.Button(frame, text = "Process\nDate", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8,
                                 command = lambda : processDate(root))
    processDateButton.place(x = 375, y = 180, anchor = tk.CENTER)

    viewAirportsButton = tk.Button(frame, text = "View\nAirports", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8,
                                  command = lambda : viewAirports(root))
    viewAirportsButton.place(x = 187.5, y = 270, anchor = tk.CENTER)

    viewAirlinesButton = tk.Button(frame, text = "View\nAirlines", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8,
                                  command = lambda : viewAirlines(root, None))
    viewAirlinesButton.place(x = 312.5, y = 270, anchor = tk.CENTER)

    viewCustomersButton = tk.Button(frame, text = "View\nCustomers", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8,
                                   command = lambda : viewCustomers(root, None))
    viewCustomersButton.place(x = 187.5, y = 360, anchor = tk.CENTER)

    viewOwnersButton = tk.Button(frame, text = "View\nOwners", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8,
                                command = lambda : viewOwners(root))
    viewOwnersButton.place(x = 312.5, y = 360, anchor = tk.CENTER)
    
    logoutButton = tk.Button(frame, text = "Logout", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8,
                            command = lambda : logout(root))
    logoutButton.place(x = 250, y = 450, anchor = tk.CENTER)

    frame.pack()
    
def customerHomeSetUp(root):
    root.title("Customer Home")
    frame = tk.Frame(root, height = 500, width = 500)
    frameList.append(frame)

    homeLabel = tk.Label(frame, text = "Customer Home", font = ("Comic Sans MS", 20))
    homeLabel.place(x = 250, y = 100, anchor = tk.CENTER)

    bookFlightButton = tk.Button(frame, text = "Book\nFlight", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8,
                            command = lambda : bookFlight(root))
    bookFlightButton.place(x = 187.5, y = 180, anchor = tk.CENTER)

    cancelFlightButton = tk.Button(frame, text = "Cancel\nFlight", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8,
                            command = lambda : cancelBookedFlight(root, None))
    cancelFlightButton.place(x = 312.5, y = 180, anchor = tk.CENTER)

    viewPropertiesButton = tk.Button(frame, text = "View\nProperties", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8)
    viewPropertiesButton.place(x = 125, y = 250, anchor = tk.CENTER)

    reservePropertyButton = tk.Button(frame, text = "Reserve\nProperty", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8,
                            command = lambda : reserveProperty(root))
    reservePropertyButton.place(x = 250, y = 250, anchor = tk.CENTER)

    cancelReservationButton = tk.Button(frame, text = "Cancel\nReservation", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8)
    cancelReservationButton.place(x = 375, y = 250, anchor = tk.CENTER)

    reviewPropertyButton = tk.Button(frame, text = "Review\nProperty", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8)
    reviewPropertyButton.place(x = 125, y = 320, anchor = tk.CENTER)

    viewReservationButton = tk.Button(frame, text = "View\nReservation", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8)
    viewReservationButton.place(x = 250, y = 320, anchor = tk.CENTER)
    
    rateOwnerButton = tk.Button(frame, text = "View\nReservation", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8)
    rateOwnerButton.place(x = 375, y = 320, anchor = tk.CENTER)
    
    logoutButton = tk.Button(frame, text = "Logout", font = ("Comic Sans MS", 10), fg = "white", bg = "black", width = 8,
                            command = lambda : logout(root))
    logoutButton.place(x = 250, y = 390, anchor = tk.CENTER)

    frame.pack()  

In [170]:
### HELPER FUNCTIONS ###

def displayTable(frame, tree, df, columns, size):
    tree["column"] = columns
    tree["show"] = "headings"
    
    scroll = ttk.Scrollbar(orient = "vertical", command = tree.yview)
    tree.configure(yscrollcommand = scroll.set)
    scroll.place(x = 483, y = 100, height = 400)
    
    col = 0
    for column in tree["column"]:
        tree.heading(column, text = column)
        if type(size) == int:
            tree.column(column, width = size)
        else:
            tree.column(column, width = size[col])
            col += 1

    df_rows = df.to_numpy().tolist()
    ind = 0
    for row in df_rows:
        tree.insert("", "end", ind, values = row)
        ind += 1
        
def destroyPage():
    frameList[0].pack_forget()
    del frameList[0]